model of a domain knowledge: atoms (Q) 
{a, b, c, d, e} 

model of knowledge space: combinatorial structure (K) 
all feasible subsets 

{a, b} {a, b, c} {d, e} 

 

current state: {a, b} 
how to make suggestions? 
look up the knowledge space for immediate superset: {a, b, c} 

1. pL – latent (mastery) 
2. pT – transition (learning) 
3. pG – guess 
4. pS – slip 
5. Learning order (K) 
6. Problem difficulty 
7. Prior knowledge 




In [ ]:
#@title Initialize Parameters

import numpy as np

def initialize_parameters(pL, pT, pS, pG):
  
  np.random.seed(1)

  pL =  np.random.randn(1)
  pT =  np.random.randn(1)
  pS =  np.random.randn(1)
  pG =  np.random.randn(1)

  parameters = {
      'pL': pL,
      'pT': pT,
      'pS': pS,
      'pG': pG
  }

  return parameters

$Equation (a): {\displaystyle p(L_{1})_{u}^{k}=p(L_{0})^{k}}$

$Equation (b): {\displaystyle p(L_{t}|obs=correct)_{u}^{k}={\frac {p(L_{t})_{u}^{k}\cdot (1-p(S)^{k})}{p(L_{t})_{u}^{k}\cdot (1-p(S)^{k})+(1-p(L_{t})_{u}^{k})\cdot p(G)^{k}}}}$

In [ ]:
#@title Eq. b

# Two state (0 or 1)
# 0 when pL < 0.5; 1 when pL >= 0.5

def correct_latent(pL, pS, pG, T):

  parameters = {}

  for t in range(T-1):
    parameters['pL_correct_obs' + str(t)] = np.dot(pL, (1 - pS))/(np.dot(pL, (1 - pS)) + np.dot((1 - pL), pG))

  return 'pL_correct_obs' + str(t)

$Equation (c): {\displaystyle p(L_{t}|obs=wrong)_{u}^{k}={\frac {p(L_{t})_{u}^{k}\cdot p(S)^{k}}{p(L_{t})_{u}^{k}\cdot p(S)^{k}+(1-p(L_{t})_{u}^{k})\cdot (1-p(G)^{k})}}}$

In [ ]:
#@title Eq. c

# Two state (0 or 1)
# 0 when pL < 0.5; 1 when pL >= 0.5

def wrong_latent(pL, pS, pG, T):

  parameters = {}

  for t in range(T-1):
    parameters['pL_wrong_obs' + str(t)] = np.dot(pL, pS)/(np.dot(pL, pS) + np.dot((1 - pL), (1 - pG)))

  return 'pL_wrong_obs' + str(t)

$Equation (d): {\displaystyle p(L_{t+1})_{u}^{k}=p(L_{t}|obs)_{u}^{k}+(1-p(L_{t}|obs)_{u}^{k})\cdot p(T)^{k}}$

In [ ]:
#@title Eq. d

def update_latent(pL_obs, pT, T, condition):

  parameters = {}

  if condition == 'correct':  
    for t in range(T-1):
      pL = 'pL_correct_obs' + str(t)
      parameters['pL' + str(t + 1)] = pL + np.multiply((1 - pL), pT)

  elif condition == 'wrong':  
    for t in range(T-1):
      pL = 'pL_wrong_obs' + str(t)
      parameters['pL' + str(t + 1)] = pL + np.multiply((1 - pL), pT)
  
  return 'pL' + str(t + 1)

$Equation (e): {\displaystyle p(C_{t+1})_{u}^{k}=p(L_{t+1})_{u}^{k}\cdot (1-p(S)^{k})+(1-p(L_{t+1})_{u}^{k})\cdot p(G)^{k}}$

In [ ]:
#@title Eq. e

def observation(pL, pS, pG, T):

  parameters = {}

  for t in range(T-1):
    parameters['pC' + str(t + 1)] = np.dot(pL, (1 - pS)) + np.dot((1 - pL), pG)

  return 'pC' + str(t + 1)

$Equation (f): {\displaystyle p(G_{t+1})^{k}=p(G_{t}|obs)^{k}+p(T)^{k}}$

In [ ]:
#@title Eq. f
#pT: prob of this item is learnable in 1t
#is this also pT: how likely the person will learn (related to personal motives, comprehension etc)

def update_guess(pG, pT, learning_rate, T):

  parameters = {}

  for t in range(T-1):
    #parameters['pG' + str(t + 1)] = pG + np.multiply(learning_rate, pT)
    parameters['pG' + str(t + 1)] = pG + pT

  assert pG < pC

  return 'pG' + str(t + 1)

$Equation (g): {\displaystyle p(S_{t+1})^{k}=p(S_{t}|obs)^{k}+p(T)^{k}}$

In [ ]:
#@title Eq. g

def update_slip(pS, pT, learning_rate, T):

  parameters = {}

  for t in range(T-1):
    #parameters['pS' + str(t + 1)] = pS - np.multiply(learning_rate, pT)
    parameters['pS' + str(t + 1)] = pS - pT

  assert pL_wrong_obs < pS

  return 'pS' + str(t + 1)

$Equation (h): {\displaystyle p(T_{t})^{k}= \frac{p(L_{t+1})_{u}^{k}-p(L_{t})_{u}^{k}}{1-p(L_{t})_{u}^{k}}}$

In [ ]:
#@title Eq. h

#learning order matters
#def pT -> [0,1] 
#forgetting for pT -> could it be [-1,1]

def update_transition(pL_obs, pT, T, condition):

  parameters = {}

  if condition == 'correct':  
    for t in range(T-1):
      pL1 = 'pL_correct_obs' + str(t)
      pL2 = 'pL_correct_obs' + str(t + 1)
      parameters['pT' + str(t)] =  np.divide((pL2 - pL1), (1 - pL1))

  elif condition == 'wrong':  
    for t in range(T-1):
      pL1 = 'pL_wrong_obs' + str(t)
      pL2 = 'pL_wrong_obs' + str(t + 1)
      parameters['pT' + str(t)] =  np.divide((pL2 - pL1), (1 - pL1))
      
  assert -1 <= 'pT' + str(t) <= 1
  
  return 'pT' + str(t)

Remaining questions: <br/>
1. reduce pS <br/>
2. how to measure program effectiveness